<h3>Statement of Purpose</h3>

The purpose of this notebook is to investigate if one or more density based clustering methods will produce any meaningful categorization of countries from flow of goods data from the United Nations. The methods used will be [DBSCAN](https://en.wikipedia.org/wiki/DBSCAN) and [meanshift](https://en.wikipedia.org/wiki/Mean_shift) clustering. The flow of good data can be downloaded or explored [here](http://data.un.org/Explorer.aspx](http://data.un.org/Explorer.aspx). The countries will be clustered based on the difference in exported and imported values aggregated acoss accross broad categories ("rubber and articles thereof"). The objective will be to determine not only if the methods yeild useful results but also if density based clustering methods are applicable to this problem and if they perform better than partition based methods.

In [1]:
#imports

import pandas as pd
import numpy as np
import matplotlib as ml
import scipy as sc
import matplotlib as mp


In [2]:
#imports to set file paths

import os
import sys
#os.environ["PYSPARK_PYTHON"] = "/opt/continuum/anaconda/bin/python"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/usr/local/spark-2.4.5-bin-hadoop2.7"
os.environ["SPARK_HOME"] = "/home/harry/spark/spark-3.1.2-bin-hadoop3.2"


In [3]:
#import spark


import pyspark
#import numpy as np
from pyspark.sql import SQLContext
from pyspark import SparkContext

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[2]").appName("commoditydata").getOrCreate()


In [5]:
#import sql functions


import pyspark.sql.functions as f

<h3>Data Preperation and Cleaning</h3>

We will first load the main data into a dataframe (df), this data is quite large for the memory of the working computer so the spark framework will be employed. The first step will be to filter the data for the correct year, 2016. Subsequently the data will be cleaned of null values and two new datframes made for import and export data on the trade values in USD, these are "dfexp" and "dfimp" respectively. These frames will then be joined to form a new dataframe "dfflow" and a difference column will be added to the dataframe that subtracts the import value from the export value. Finally the columns for country, trade in usd, and category will be taken and pivoted so that and average USD trade value per country per category is established. Here the trade categoies will be columns, the countries the first column and the average values in USD entries.

In [6]:
#read dataframe


df = spark.read.options(header = 'true',  inferSchema='True', delimiter=',').csv('/home/harry/exports/commodity_trade_statistics_data.csv')

In [7]:
df.show()

+---------------+----+---------+--------------------+------+---------+---------+---------------+-----------+---------------+
|country_or_area|year|comm_code|           commodity|  flow|trade_usd|weight_kg|  quantity_name|   quantity|       category|
+---------------+----+---------+--------------------+------+---------+---------+---------------+-----------+---------------+
|    Afghanistan|2016|   010410|         Sheep, live|Export|     6088|     2339|Number of items|       51.0|01_live_animals|
|    Afghanistan|2016|   010420|         Goats, live|Export|     3958|      984|Number of items|       53.0|01_live_animals|
|    Afghanistan|2008|   010210|Bovine animals, l...|Import|  1026804|      272|Number of items|     3769.0|01_live_animals|
|        Albania|2016|   010290|Bovine animals, l...|Import|  2414533|  1114023|Number of items|     6853.0|01_live_animals|
|        Albania|2016|   010392|Swine, live excep...|Import| 14265937|  9484953|Number of items|    96040.0|01_live_animals|


In [8]:

df.printSchema()

root
 |-- country_or_area: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- comm_code: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- flow: string (nullable = true)
 |-- trade_usd: long (nullable = true)
 |-- weight_kg: long (nullable = true)
 |-- quantity_name: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- category: string (nullable = true)



In [9]:
df.schema.names

['country_or_area',
 'year',
 'comm_code',
 'commodity',
 'flow',
 'trade_usd',
 'weight_kg',
 'quantity_name',
 'quantity',
 'category']

In [10]:
df.drop(df.comm_code).printSchema()

root
 |-- country_or_area: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- commodity: string (nullable = true)
 |-- flow: string (nullable = true)
 |-- trade_usd: long (nullable = true)
 |-- weight_kg: long (nullable = true)
 |-- quantity_name: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- category: string (nullable = true)



In [11]:
df.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+---------------+----+---------+---------+----+---------+---------+-------------+--------+--------+
|country_or_area|year|comm_code|commodity|flow|trade_usd|weight_kg|quantity_name|quantity|category|
+---------------+----+---------+---------+----+---------+---------+-------------+--------+--------+
|              0| 726|      726|      726|1452|     1452|   129927|         1452|  306309|    1452|
+---------------+----+---------+---------+----+---------+---------+-------------+--------+--------+



In [12]:
df.filter(df.year == 2016).select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+---------------+----+---------+---------+----+---------+---------+-------------+--------+--------+
|country_or_area|year|comm_code|commodity|flow|trade_usd|weight_kg|quantity_name|quantity|category|
+---------------+----+---------+---------+----+---------+---------+-------------+--------+--------+
|              0|   0|        0|        0|  61|       61|     6938|           61|   15206|      61|
+---------------+----+---------+---------+----+---------+---------+-------------+--------+--------+



In [13]:
#df = df.na.drop()

df.filter(df.category == 'all_commodities').select('category').distinct().show()

+---------------+
|       category|
+---------------+
|all_commodities|
+---------------+



In [14]:
#Create the export dataframe.

dfexp = df.filter((df.year == 2016) & (df.flow == 'Export'))

dfexpu = df.filter((df.year == 2016) & (df.flow == 'Export') & (df.quantity_name != 'No Quantity') & (df.quantity_name.isNotNull())).drop("comm_code", "year")

dfexp = dfexp.withColumn('cost_per_kg', dfexp['trade_usd']/dfexp['weight_kg'])

dfexpu = dfexp.withColumn('cost_per_kg', dfexp['trade_usd']/dfexp['weight_kg'])

In [15]:
#Create the import dataframe.

dfimp = df.filter((df.year == 2016) & (df.flow == 'Import'))


dfimpu = df.filter((df.year == 2016) & (df.flow == 'Import') & (df.quantity_name != 'No Quantity') & (df.quantity_name.isNotNull())).drop("comm_code", "year")

dfimp = dfimp.withColumn('cost_per_kg', dfimp['trade_usd']/dfimp['weight_kg'])

dfimpu = dfimp.withColumn('cost_per_kg', dfimp['trade_usd']/dfimp['weight_kg'])

In [16]:

#Now we perform the join using an outer join function on the import and export dataframes. 



#dfflow = dfexp.select('country_or_area', 'commodity', 'category', f.col('trade_usd').alias('trade_usd_exp'), f.col('weight_kg').alias('weight_kg_exp')).join(dfimp.select('country_or_area', 'commodity', 'category', f.col('trade_usd').alias('trade_usd_imp'), f.col('weight_kg').alias('weight_kg_imp')), ['country_or_area', 'commodity', 'category'], 'outer')


dfflow = dfexp.select('country_or_area', 'commodity', 'category', 'quantity_name', f.col('trade_usd').alias('trade_usd_exp'), f.col('weight_kg').alias('weight_kg_exp'), f.col('quantity').alias('quantity_exp')).join(dfimp.select('country_or_area', 'commodity', 'category', 'quantity_name', f.col('trade_usd').alias('trade_usd_imp'), f.col('weight_kg').alias('weight_kg_imp'), f.col('quantity').alias('quantity_imp')), ['country_or_area', 'commodity', 'category', 'quantity_name'], 'outer')

dfflowu = dfexpu.select('country_or_area', 'commodity', 'category', 'quantity_name', f.col('trade_usd').alias('trade_usd_exp'), f.col('weight_kg').alias('weight_kg_exp'), f.col('quantity').alias('quantity_exp')).join(dfimpu.select('country_or_area', 'commodity', 'category', 'quantity_name', f.col('trade_usd').alias('trade_usd_imp'), f.col('weight_kg').alias('weight_kg_imp'), f.col('quantity').alias('quantity_imp')), ['country_or_area', 'commodity', 'category', 'quantity_name'], 'outer')





In [17]:
dfflow.filter(dfflow.category == 'all_commodities').select('category').distinct().show()

+---------------+
|       category|
+---------------+
|all_commodities|
+---------------+



In [18]:
#We will create columns the give a net flow of goods in terms of currency in USD, in kg, and finally in a normalized price per kilogram.


dfflow = dfflow.withColumn("difference_usd", dfflow['trade_usd_imp'] - dfflow['trade_usd_exp']).withColumn("difference_kg", dfflow['weight_kg_imp'] - dfflow['weight_kg_exp']).withColumn("difference_usd_kg", (dfflow['trade_usd_imp'] - dfflow['trade_usd_exp'])/(dfflow['weight_kg_imp'] - dfflow['weight_kg_exp']))




In [19]:
dfflowu = dfflowu.withColumn("difference_usd", dfflowu['trade_usd_imp'] - dfflowu['trade_usd_exp']).withColumn("difference_kg", dfflowu['weight_kg_imp'] - dfflowu['weight_kg_exp']).withColumn("difference_usd_kg", (dfflowu['trade_usd_imp'] - dfflowu['trade_usd_exp'])/(dfflowu['weight_kg_imp'] - dfflowu['weight_kg_exp'])).withColumn("difference_usd", dfflowu['trade_usd_imp'] - dfflowu['trade_usd_exp']).withColumn("difference_kg", dfflowu['weight_kg_imp'] - dfflowu['weight_kg_exp']).withColumn("difference_usd_unit", (dfflowu['trade_usd_imp'] - dfflowu['trade_usd_exp'])/(dfflowu['quantity_imp'] - dfflowu['quantity_exp']))


In [20]:
#Eliminate white space and special characters from the target columns

dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', ' ', '_'))
dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', '-', '_'))
dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', '<', 'lt'))
dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', '>', 'gt'))
dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', '"', ''))
dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', '#', ''))
dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', ':', ''))
dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', ',', ''))
dfflow = dfflow.withColumn('commodity', f.regexp_replace('commodity', '\\.', ''))
dfflow = dfflow.withColumn('category', f.regexp_replace('category', ' ', '_'))
#dfflow = dfflow.withColumn('quantity_name', f.regexp_replace('quantity_name', ' ', '_'))

In [21]:
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', ' ', '_'))
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', '-', '_'))
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', '<', 'lt'))
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', '>', 'gt'))
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', '"', ''))
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', '#', ''))
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', ':', ''))
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', ',', ''))
dfflowu = dfflowu.withColumn('commodity', f.regexp_replace('commodity', '\\.', ''))
dfflowu = dfflowu.withColumn('category', f.regexp_replace('category', ' ', '_'))
dfflowu = dfflowu.withColumn('quantity_name', f.regexp_replace('quantity_name', ' ', '_'))

In [22]:
#dfflow.show()

In [23]:
dfflow.printSchema()

root
 |-- country_or_area: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity_name: string (nullable = true)
 |-- trade_usd_exp: long (nullable = true)
 |-- weight_kg_exp: long (nullable = true)
 |-- quantity_exp: double (nullable = true)
 |-- trade_usd_imp: long (nullable = true)
 |-- weight_kg_imp: long (nullable = true)
 |-- quantity_imp: double (nullable = true)
 |-- difference_usd: long (nullable = true)
 |-- difference_kg: long (nullable = true)
 |-- difference_usd_kg: double (nullable = true)



In [24]:
dfflowu.printSchema()

root
 |-- country_or_area: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity_name: string (nullable = true)
 |-- trade_usd_exp: long (nullable = true)
 |-- weight_kg_exp: long (nullable = true)
 |-- quantity_exp: double (nullable = true)
 |-- trade_usd_imp: long (nullable = true)
 |-- weight_kg_imp: long (nullable = true)
 |-- quantity_imp: double (nullable = true)
 |-- difference_usd: long (nullable = true)
 |-- difference_kg: long (nullable = true)
 |-- difference_usd_kg: double (nullable = true)
 |-- difference_usd_unit: double (nullable = true)



In [25]:

#df1 = dfflow.select('country_or_area', 'category', 'trade_usd').groupBy('country_or_area').pivot('commodity').avg('trade_usd')

df2 = dfflow.select('country_or_area', 'category', 'difference_usd_kg').groupBy('country_or_area').pivot('category').avg('difference_usd_kg')

#df3 = dfflow.select('country_or_area', 'category', 'difference_usd_kg').groupby('country_or_area').pivot('category').avg('difference_usd_kg')



df3 = dfflow.select('country_or_area', 'category', 'difference_usd').groupby('country_or_area').pivot('category').avg('difference_usd')

df3u = dfflowu.select('country_or_area', 'category', 'difference_usd_unit').groupby('country_or_area').pivot('category').avg('difference_usd_unit')



In [26]:
#df3.printSchema()

In [27]:
df3.select(f.count(f.when(df3.all_commodities.isNull() == True, df3.all_commodities)), f.count(f.when(df3.all_commodities == 0, df3.all_commodities))).show()

+----------------------------------------------------------------------------+---------------------------------------------------------------+
|count(CASE WHEN ((all_commodities IS NULL) = true) THEN all_commodities END)|count(CASE WHEN (all_commodities = 0) THEN all_commodities END)|
+----------------------------------------------------------------------------+---------------------------------------------------------------+
|                                                                           0|                                                              0|
+----------------------------------------------------------------------------+---------------------------------------------------------------+



In [28]:
#The USD trade values are normalized against the absolute value of all traded commodities.

for i in df3.columns[1:len(df3.columns)]:
    df3 = df3.withColumn(i, f.col(i)/f.abs('all_commodities'))
    
    


In [29]:
df2 = df2.drop('all_commodities').drop('99_commodities_not_specified_according_to_kind')
df2 = df2.fillna(value = 0)
df2 = df2.na.fill(value = 0)

In [30]:
#Null values are filled and appropriate columns dropped.
df3 = df3.drop('all_commodities').drop('99_commodities_not_specified_according_to_kind')
df3 = df3.fillna(value = 0)
df3 = df3.na.fill(value = 0)

In [31]:
#Null values are filled and appropriate columns dropped.
df3u = df3u.drop('all_commodities').drop('99_commodities_not_specified_according_to_kind')
df3u = df3u.fillna(value = 0)
df3u = df3u.na.fill(value = 0)

In [32]:
#Dataframes are converted to pandas.

pdf2 = df2.toPandas()

pdf3 = df3.toPandas()

pdf3u = df3u.toPandas()


In [33]:
pdf3.head()

,country_or_area,01_live_animals,02_meat_and_edible_meat_offal,03_fish_crustaceans_molluscs_aquatic_invertebrates_ne,04_dairy_products_eggs_honey_edible_animal_product_nes,05_products_of_animal_origin_nes,06_live_trees_plants_bulbs_roots_cut_flowers_etc,07_edible_vegetables_and_certain_roots_and_tubers,08_edible_fruit_nuts_peel_of_citrus_fruit_melons,09_coffee_tea_mate_and_spices,...,88_aircraft_spacecraft_and_parts_thereof,89_ships_boats_and_other_floating_structures,90_optical_photo_technical_medical_etc_apparatus,91_clocks_and_watches_and_parts_thereof,92_musical_instruments_parts_and_accessories,93_arms_and_ammunition_parts_and_accessories_thereof,94_furniture_lighting_signs_prefabricated_buildings,95_toys_games_sports_requisites,96_miscellaneous_manufactured_articles,97_works_of_art_collectors_pieces_and_antiques
0,Rep. of Moldova,-0.000137,0.0,0.000000,0.000000,0.000659,0.000652,0.0,0.0,0.00000,...,0.000067,0.000012,0.0,0.0,0.000028,0.000000,0.0,0.000000,0.0,-0.000006
1,Paraguay,-0.000273,0.0,0.000000,0.000649,-0.010632,0.000070,0.0,0.0,0.00000,...,0.000321,-0.000444,0.0,0.0,0.000042,0.000000,0.0,0.000000,0.0,0.000018
2,Senegal,0.000001,0.0,0.000000,0.000000,-0.000014,-0.000055,0.0,0.0,0.00000,...,-0.000057,0.000504,0.0,0.0,-0.000006,-0.000068,0.0,0.000000,0.0,-0.000086
3,Cabo Verde,0.000000,0.0,-0.009569,0.000000,0.000000,0.000000,0.0,0.0,0.00227,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,-0.000083,0.0,0.000000
4,Sweden,0.000446,0.0,0.000000,0.000000,0.003710,0.030416,0.0,0.0,0.00000,...,0.015655,-0.002314,0.0,0.0,-0.000067,-0.000174,0.0,0.000000,0.0,0.003978


In [34]:
pdf3.describe()

,01_live_animals,02_meat_and_edible_meat_offal,03_fish_crustaceans_molluscs_aquatic_invertebrates_ne,04_dairy_products_eggs_honey_edible_animal_product_nes,05_products_of_animal_origin_nes,06_live_trees_plants_bulbs_roots_cut_flowers_etc,07_edible_vegetables_and_certain_roots_and_tubers,08_edible_fruit_nuts_peel_of_citrus_fruit_melons,09_coffee_tea_mate_and_spices,10_cereals,...,88_aircraft_spacecraft_and_parts_thereof,89_ships_boats_and_other_floating_structures,90_optical_photo_technical_medical_etc_apparatus,91_clocks_and_watches_and_parts_thereof,92_musical_instruments_parts_and_accessories,93_arms_and_ammunition_parts_and_accessories_thereof,94_furniture_lighting_signs_prefabricated_buildings,95_toys_games_sports_requisites,96_miscellaneous_manufactured_articles,97_works_of_art_collectors_pieces_and_antiques
count,122.000000,122.000000,122.000000,1.220000e+02,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,...,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000,122.000000
mean,-0.001053,-0.011920,-0.002613,-2.083358e-03,0.000085,-0.000882,-0.001247,-0.003913,0.000575,0.010036,...,0.005990,0.001385,0.001395,0.000569,0.000431,0.002081,0.004613,0.002957,0.000431,0.000452
std,0.009165,0.132223,0.017968,1.970160e-02,0.006157,0.013955,0.012758,0.024637,0.009021,0.253750,...,0.030163,0.012930,0.014560,0.004280,0.003291,0.015574,0.045293,0.027572,0.003546,0.005114
min,-0.095846,-1.459185,-0.173121,-1.619456e-01,-0.038084,-0.144381,-0.140359,-0.222709,-0.024299,-1.580153,...,-0.038098,-0.050580,-0.000492,-0.000511,-0.001139,-0.005454,-0.016121,-0.002993,-0.000511,-0.007743
25%,-0.000468,0.000000,0.000000,0.000000e+00,-0.000026,-0.000002,0.000000,0.000000,0.000000,-0.000778,...,-0.000026,-0.000289,0.000000,0.000000,0.000000,-0.000004,0.000000,0.000000,0.000000,-0.000035
50%,0.000000,0.000000,0.000000,2.994346e-07,0.000000,0.000025,0.000000,0.000000,0.000000,0.000796,...,0.000192,0.000009,0.000000,0.000000,0.000020,0.000000,0.000000,0.000000,0.000000,0.000005
75%,0.000158,0.000633,0.000000,8.980635e-04,0.000111,0.000359,0.000000,0.000000,0.000295,0.004358,...,0.003097,0.000802,0.000000,0.000040,0.000060,0.000035,0.000342,0.000230,0.000051,0.000073
max,0.013960,0.014993,0.004004,1.846240e-02,0.051081,0.034107,0.001904,0.005749,0.094785,2.163247,...,0.297084,0.088134,0.160778,0.045087,0.035505,0.128174,0.499243,0.304253,0.038995,0.055185


<h3>Modeling</h3>

   The data will be modeled using DBSCAN and meanshift methods. These are density based methods which attempt to find localities with the highest density of points [1](https://en.wikipedia.org/wiki/Cluster_analysis#Density-based_clustering). These methods are not available natively through spark so the sci-kit learn versions will be used necessitating the transformation of the spark dataframes to pandas ones. Two functions will be constructed to take appropriate parameters allowing the use of loops to search for optimal combinations of parameters. The labels of the returned from the function using the optimized parameters will then be used to produce cluster labels which will be assigned to the previous dataframes.

In [37]:
#Create a DBSCAN function.


from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.preprocessing import StandardScaler


def dbscans(data, eps, mn):

    X = StandardScaler().fit_transform(data)

    # #############################################################################
    # Compute DBSCAN
    db = DBSCAN(eps=eps, min_samples=mn).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    if n_clusters_ > 1:
        #print("Silhouette Coefficient: %epsf" % metrics.silhouette_score(X, labels))
        return [eps, mn, n_clusters_, n_noise_, metrics.silhouette_score(X, labels)], labels
    else:
        #return [eps, mn, n_clusters, n_noise, metrics.silhouette_score(X, labels)], labels
        return [eps, mn, n_clusters_, n_noise_], labels

In [39]:
#Use a combination of epsilon and min_sample values to get the best combination

scores = []
#data = pdf3.as_matrix(columns = pdf3.columns[1:])
data = pdf3.iloc[:,1:].values
#print(range(.2,.6,.1))

for i in np.arange(1.5,2.5,.01):
    for k in range(1,122,1):
        db =  dbscans(data,i,k)
        scores.append(db[0])

In [40]:
scs = pd.DataFrame([i for i in scores if len(i) > 4], columns = ['eps', 'min_samples', 'n_clusters', 'n_noise', 'silhouette'])
scs.head()

,eps,min_samples,n_clusters,n_noise,silhouette
0,1.50,1,48,0,0.242462
1,1.50,2,2,46,0.169259
2,1.51,1,48,0,0.242462
3,1.51,2,2,46,0.169259
4,1.52,1,48,0,0.242462


In [41]:
print(max(scs.n_clusters))

48


In [42]:
cs = scs.nlargest(20, 'silhouette')

In [43]:
cs.head()

,eps,min_samples,n_clusters,n_noise,silhouette
157,2.30,1,31,0,0.3425
158,2.31,1,31,0,0.3425
159,2.32,1,31,0,0.3425
160,2.33,1,31,0,0.3425
161,2.34,1,31,0,0.3425


In [44]:
dbbest = dbscans(data, 2.30, 1)

In [45]:
pdf3l = pdf3.assign(labels = dbbest[1])
#pdf3.insert(len(pdf3.columns) - 1, 'labels', dbbest[1])

In [46]:
pdf3l[['country_or_area', 'labels']].head(25)

,country_or_area,labels
0,Rep. of Moldova,0
1,Paraguay,1
2,Senegal,0
3,Cabo Verde,0
4,Sweden,2
5,Guyana,3
6,Philippines,0
7,Malaysia,0
8,Singapore,0
9,Fiji,0


In [47]:
pdf3l['labels'].value_counts()

0     92
16     1
29     1
28     1
27     1
26     1
25     1
24     1
23     1
22     1
21     1
20     1
19     1
18     1
17     1
15     1
1      1
14     1
13     1
12     1
11     1
10     1
9      1
8      1
7      1
6      1
5      1
4      1
3      1
2      1
30     1
Name: labels, dtype: int64

In [48]:
#We create a meanshift function.

from sklearn.cluster import MeanShift, estimate_bandwidth

def meanshifts(data, qntl, binseed):
    bandwidth = estimate_bandwidth(data, quantile=qntl)
    if bandwidth > 0:
        
        ms = MeanShift(bandwidth=bandwidth, bin_seeding=binseed)
        ms.fit(data)
        labels = ms.labels_
        cluster_centers = ms.cluster_centers_
        labels_unique = np.unique(labels)
        n_clusters_ = len(labels_unique)
        sil = metrics.silhouette_score(data, labels)
        return [qntl ,sil, labels, cluster_centers, labels_unique, n_clusters_]
    else:
        pass


In [49]:
scoresm = []
#data = pdf3.as_matrix(columns = pdf3.columns[1:])
data = pdf3.iloc[:,1:].values
#print(range(.2,.6,.1))

for i in np.arange(.05,1,.05):
    msh =  meanshifts(data, i, True)
    #print(i)
    scoresm.append(msh)

In [50]:
mss = pd.DataFrame(scoresm, columns = ['qrtl', 'sil', 'labels', 'cluster_centers', 'labels_unique', 'n_clusters_'])

In [51]:
mss.head()

,qrtl,sil,labels,cluster_centers,labels_unique,n_clusters_
0,0.05,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7
1,0.10,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7
2,0.15,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7
3,0.20,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7
4,0.25,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7


In [52]:
mssl = mss.nlargest(20,'sil')

In [53]:
mssl.head(20)

,qrtl,sil,labels,cluster_centers,labels_unique,n_clusters_
18,0.95,0.866348,"[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 3, ...","[[-0.00037264184333729924, -0.0001540965427458...","[0, 1, 2, 3, 4]",5
17,0.90,0.761613,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0004923032269844726, 0.000156147279909072...","[0, 1, 2, 3, 4, 5, 6]",7
16,0.85,0.759167,"[0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6, 7]",8
15,0.80,0.759167,"[0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6, 7]",8
13,0.70,0.759167,"[0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6, 7]",8
10,0.55,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7
14,0.75,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7
12,0.65,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7
11,0.60,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7
0,0.05,0.745975,"[0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, ...","[[-0.0002985000388455241, 0.000157554012160505...","[0, 1, 2, 3, 4, 5, 6]",7


In [54]:
pdf3m = pdf3.assign(labels = mssl.labels[1])

In [55]:
print(mssl.labels[1])

[0 0 0 0 0 6 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 1 0 0 0 0 2 0
 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 6 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]


In [56]:
pdf3m[['country_or_area', 'labels']].head()

,country_or_area,labels
0,Rep. of Moldova,0
1,Paraguay,0
2,Senegal,0
3,Cabo Verde,0
4,Sweden,0


In [57]:
pdf3m['labels'].value_counts()

0    114
6      3
4      1
5      1
1      1
2      1
3      1
Name: labels, dtype: int64

In [58]:
pdf3m[pdf3m['labels'] == 6].describe()

,01_live_animals,02_meat_and_edible_meat_offal,03_fish_crustaceans_molluscs_aquatic_invertebrates_ne,04_dairy_products_eggs_honey_edible_animal_product_nes,05_products_of_animal_origin_nes,06_live_trees_plants_bulbs_roots_cut_flowers_etc,07_edible_vegetables_and_certain_roots_and_tubers,08_edible_fruit_nuts_peel_of_citrus_fruit_melons,09_coffee_tea_mate_and_spices,10_cereals,...,89_ships_boats_and_other_floating_structures,90_optical_photo_technical_medical_etc_apparatus,91_clocks_and_watches_and_parts_thereof,92_musical_instruments_parts_and_accessories,93_arms_and_ammunition_parts_and_accessories_thereof,94_furniture_lighting_signs_prefabricated_buildings,95_toys_games_sports_requisites,96_miscellaneous_manufactured_articles,97_works_of_art_collectors_pieces_and_antiques,labels
count,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.0
mean,-0.028208,-0.485796,-0.003776,-0.048745,0.004332,0.007839,-0.046786,-0.015478,0.031837,-0.342370,...,0.028516,0.053593,0.015138,0.014557,0.036593,0.166414,0.101963,0.012998,0.017828,6.0
std,0.059168,0.842980,0.006540,0.098037,0.044740,0.023361,0.081036,0.026809,0.054516,1.151202,...,0.051663,0.092825,0.025937,0.018559,0.068032,0.288238,0.175189,0.022514,0.032364,0.0
min,-0.095846,-1.459185,-0.011328,-0.161946,-0.038084,-0.010609,-0.140359,-0.046434,0.000000,-1.580153,...,-0.003116,0.000000,0.000000,0.000171,-0.005360,0.000000,0.000000,0.000000,-0.001732,6.0
25%,-0.049292,-0.729593,-0.005664,-0.077356,-0.019042,-0.005296,-0.070180,-0.023217,0.000363,-0.861688,...,-0.001293,0.000000,0.000164,0.004083,-0.002654,0.000000,0.000819,0.000000,-0.000850,6.0
50%,-0.002739,0.000000,0.000000,0.007234,-0.000001,0.000018,0.000000,0.000000,0.000726,-0.143223,...,0.000530,0.000000,0.000327,0.007994,0.000051,0.000000,0.001638,0.000000,0.000032,6.0
75%,0.005610,0.000899,0.000000,0.007855,0.025540,0.017062,0.000000,0.000000,0.047755,0.276522,...,0.044332,0.080389,0.022707,0.021750,0.057569,0.249622,0.152945,0.019497,0.027609,6.0
max,0.013960,0.001798,0.000000,0.008476,0.051081,0.034107,0.000000,0.000000,0.094785,0.696267,...,0.088134,0.160778,0.045087,0.035505,0.115088,0.499243,0.304253,0.038995,0.055185,6.0


<h3>Result</h3>
        Both the DBSCAN and meanshift clustering yeilded similar to the partition based methods with singular countries placed in outlying groups while the rest were clustered together. The number of unique labels differed dramatcally between the meanshift and DBSCAN methods with the optimized DBSCAN giving 31 unique labeled clusters and the meanshift giving 5 total clusters. This discrepancy could result from poor tuning of parameters or data cleaning. However given the consistency with which different clustering methods have produced a single large cluster of countries it appears that there are simply a small number of outlying countries whose flow of goods is unique for some reason. It seems the next step would be to identify which countries are consistently shown as outliers and to find out what makes those countries unique.